In [9]:
from cabinet_robot.joint_estimation import FGJointEstimator, sturm_twist_estimation, EstimationResults
import jax
import numpy as np
from jaxlie import SE3 as jaxlie_SE3

%reload_ext autoreload
%autoreload 2

In [3]:
# jax.config.update("jax_disable_jit", True)


In [10]:
original_part_pose = np.eye(4)
original_part_pose[2, 3] = 0.1

part_poses = [np.copy(original_part_pose)]
for _ in range(10):
    original_part_pose[0, 3] += 0.01
    original_part_pose[1, 3] += 0.005
    part_poses.append(np.copy(original_part_pose))


In [11]:
estimator = FGJointEstimator()


In [13]:
estimator._build_graph(5) # builds but does not compile so should run fast


HUBER_DELTA = DeviceArray(230.92856, dtype=float32)
type(joint_parameters_variable) = <class 'articulation_estimation.factor_graph.states.GeneralJointParametersVariable'>


In [20]:
estimator.get_compiled_graph(11) # builds and compiles so should run slow the first time

reusing compiled graph for 11 samples


In [19]:

estimation = estimator.estimate_joint_twist(part_poses)
print(f"{estimation.twist=}")
print(f"{estimation.twist_frame_in_base_pose=}")

reusing compiled graph for 11 samples
Using VMAP to optimize all factor graph version
all_costs = DeviceArray([1.4655766e+00, 1.1198006e+00, 4.4433222e+00, 5.7437098e-01,
             1.0677587e+00, 3.2567003e+00, 2.2498279e+00, 3.8528821e-01,
             5.9861779e-01, 1.5201143e+00, 1.3987130e+00, 1.5672419e+00,
             1.7153782e+00, 2.9746876e+00, 1.9291578e-10, 2.3339765e+00,
             1.8845421e-01, 1.6275932e+01, 1.9859222e+00, 7.3052108e-01,
             1.8535779e+00, 1.3310499e+00, 3.0486526e+00, 1.9833050e+00,
             1.4823640e-03, 1.5397809e+00, 1.2866598e-04, 6.9169289e-01,
             3.2981355e+00, 2.4283152e-05, 3.0521581e+00, 2.1670644e+00,
             2.8607643e+00, 1.0501614e-04, 6.2815239e-08, 1.1384237e+00,
             1.5463040e+00, 2.1619160e+00, 9.6180648e-01, 5.8402424e-04,
             2.8469574e+00, 5.4321857e-04, 1.0741606e-01, 4.0930950e-05,
             2.1531644e+00, 3.7909180e-01, 6.3745821e-11, 1.4891479e+00,
             2.4112449e+00

In [21]:
# test: latent_pose second[0] = base_transform @ exp(twist * joint_state[0])

print(
    estimation.twist_frame_in_base_pose.as_matrix()
    @ jaxlie_SE3.exp(estimation.twist * estimation.current_joint_configuration).as_matrix()
)
print(
    estimation.twist_frame_in_base_pose.as_matrix()
    @ jaxlie_SE3.exp(estimation.twist * estimation.aux_data["joint_states"][0]).as_matrix()
)

import spatialmath.base as sm

# twist_in_poses_frame = sm.tr2adjoint(np.asarray(results.base_transform.as_matrix())) @ np.asarray(results.twist)
# print(jaxlie_SE3.exp(twist_in_poses_frame * results.aux_data["joint_states"][5]).as_matrix())


[[ 1.0000000e+00  3.7796130e-10 -8.7015110e-11  9.9999994e-02]
 [-3.7796130e-10  1.0000000e+00 -4.3322939e-11  4.9999997e-02]
 [ 8.7015110e-11  4.3322939e-11  1.0000000e+00  1.0000002e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[ 1.0000000e+00 -3.7787684e-10  8.7282133e-11 -3.7252903e-09]
 [ 3.7787684e-10  1.0000000e+00  4.3462203e-11 -1.8626451e-09]
 [-8.7282133e-11 -4.3462203e-11  1.0000000e+00  1.0000002e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [6]:
print(estimation.aux_data["joint_states"])

[[0.56120133]
 [0.5690947 ]
 [0.5769883 ]
 [0.58488166]
 [0.59277517]
 [0.60066855]
 [0.608562  ]
 [0.6164555 ]
 [0.6243489 ]
 [0.6322424 ]
 [0.6401358 ]]


## Explore methods for joint configuration estimation

In [7]:

#  part pose = = base_transform @ exp(twist * joint_state)
# so joint_state = log(base_transform^-1 @ part_pose) / twist
# but this last division can suffer from numerical issues...

part_pose = np.asarray(estimation.aux_data["latent_poses"]["second"][-1].as_matrix())
base_transform = np.asarray(estimation.twist_frame_in_base_pose.as_matrix())
twist = np.asarray(estimation.twist)

print(f"the GT joint configuration is {estimation.current_joint_configuration}")


the GT joint configuration is 0.6401358246803284


In [8]:
# naive joint configuration estimation -> numerical errors can dominate 
import spatialmath.base as sm

pose_in_twist_frame = sm.trinv(base_transform) @ part_pose
print(pose_in_twist_frame)
joint_state = sm.trlog(pose_in_twist_frame, twist=True,check=False) / twist
print(joint_state)
print(f"naive estimation = {np.mean(joint_state)}")

[[ 9.9994844e-01 -1.0153786e-02 -1.7876686e-04  8.0883688e-01]
 [ 1.0153701e-02  9.9994832e-01 -4.7073461e-04  4.0969858e-01]
 [ 1.8353737e-04  4.6889522e-04  9.9999988e-01  1.7051399e-04]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[ 0.64007269  0.64038825 -0.14997943  0.61411027  0.61432979  0.61415831]
naive estimation = 0.4955133144422448


In [9]:
# so weigh every factor according to the relative magnitude of the part twist to make a more robust estimate
pose_twist = sm.trlog(pose_in_twist_frame, twist=True,check=False)
normalized_pose_twist = pose_twist / np.linalg.norm(pose_twist,ord=1)
print(pose_twist/twist)
print(np.abs(normalized_pose_twist))
print(f"more robust estimation = {np.sum(pose_twist/twist * np.abs(normalized_pose_twist))}")

[ 0.64007269  0.64038825 -0.14997943  0.61411027  0.61432979  0.61415831]
[6.60724196e-01 3.30471011e-01 1.03260094e-06 3.82808736e-04
 1.47604103e-04 8.27334720e-03]
more robust estimation = 0.6399480214533965


In [10]:
# but even better method is required...
# do some explicit oultier detection or something?? 
# best option might be to use the results of the FG (which is however only valid in int the Twist frame of the FG..)

### Can we get rid of the 'twist frame' by expressing the twist in the base frame?

In [11]:
# Twist_expressed_in_base = sm.tr2adjoint(base_transform) @ twist
twist_expressed_in_base = sm.tr2adjoint(base_transform) @ twist
print(twist_expressed_in_base)
print(twist)

[ 1.26735039e+00  6.32588466e-01 -4.27035781e-05  7.65046551e-04
 -2.94882544e-04  1.65330638e-02]
[ 1.2669029e+00  6.3334805e-01 -8.4499325e-06  7.6504663e-04
 -2.9488269e-04  1.6533064e-02]


In [12]:
# now the twist is expressed in the base frame
# part_pose = part_pose[0] @ exp(twist * joint_state')


In [13]:
import rerun 
rerun.init("test-joint-estimation", spawn=True)


2023-02-27T10:17:08.329424Z  WARN rerun::run: Failed to bind address "0.0.0.0:9876". Another Rerun instance is probably running.


In [14]:
rerun.log_points(
    "part_poses",
    positions=np.array(part_poses)[:, :3, 3],
    colors=np.zeros((len(part_poses), 3), dtype=np.uint8),
    radii=0.01,
)

In [26]:
from cabinet_robot.visualisation import visualize_estimation
visualize_estimation(estimation)

(11, 4, 4)
